In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import stock_module as sm
import random
import os
import datetime
import time

-- BUY SINYALI SONRASI:

----- MA_14 MA_200'ün altında veya üzerinde ama çok yakınsa (ne kadar yakın bunu belirle)

----- MA_200 negatif olsa bile, pozitif olduktan sonra al!

----- CH14, CH200 ikisi de pozitifse; CH14 ivmesi CH200'ü geçtiğinde AL veya CH200 negatifse (geçmiş 100 dönem ile kıyasla ve düşük negatifse pozitif gibi değerlendirebilirsin), pozitife geçince al

-- Sell sinyallerinde; 

----- MA_14 MA_200 üstünde ise ve ivmeler yavaşlamıyorsa sat (ivmelerin nasıl yavaşladığını tespit et!)

----- CH14 ivmesine bak ve geçmiş ile kıyasla. Geçmiş ortalaması ile kıyasla.

----- Üç ivme birden negatife dönerse kesin sat (3 gün arka arkaya).

In [1516]:
def get_volatility(stock_list, start_date, end_date):
    volatilityTable = []
    for stock in stock_list:
        volatility = sm.volatility(stock, start_date, end_date)
        volatilityTable.append(volatility)
    
    frame = {"stock_name": stock_list, "volatility": volatilityTable} 
    frame = pd.DataFrame(frame) 
    frame = frame.dropna()
    
    return frame

In [1517]:
def whole_frame(stock_name:str, MA_days:list, start_date:str, end_date:str, RSI_day:int, trajectory_day:int):
    all_stock_value = sm.values_from_database()
    frame = sm.all_movingAverages(all_stock_value, stock_name, MA_days, start_date, end_date)
    frame = sm.moving_average_trajectory(frame, trajectory_day)
    
    RSI = sm.RSI_indicator(stock_name, RSI_day, start_date, end_date)
    RSI = RSI[["RSI"]]
    
    DD = sm.drawdowns(stock_name, start_date)
    DD = DD[["Drawdown"]]
    
    MV = sm.get_market_values(stock_name, start_date)[["marketV_to_bookV"]]
    
    stock = sm.value_stock(stock_name, start_date)
    
    frame = frame.join(RSI)
    frame = frame.join(DD)
    frame = frame.join(MV)
    frame = frame.join(stock)
    
    frame = frame.rename(columns={"HGDG_KAPANIS": "StockValue"})
    
    return frame

In [1518]:
def return_cum(returns):
    returns["product"] = returns["ret"] + 1
    print(returns["product"].cumprod()[-1])

In [1580]:
def return_strategy(frame):
    frame['day'] = np.where((frame['MA_7'] > frame['MA_50']) & 
                        (frame['MA_7'] > frame['MA_200']) &
                        (frame['MA_200_CH'] > 0), "BUY", "SELL")

    try:
        first_oberv = frame["day"][0]
        frame["check"][0] = first_oberv
        frame["check"] = frame["day"] == frame["check"] 
        frame["check_2"] = frame["check"].shift(-1)
        frame["check_3"] = frame["check"] == frame["check_2"] 
        frame['check_3'] = np.where(frame["check_3"] == False, "ok", "nope")
        frame = frame[frame["check_3"] == "ok"]
        frame = frame.loc[frame["day"] == "BUY"]
        frame['day'] = np.where(frame['check'] == False, "BUY", "SELL")
        frame = frame.drop(['check', 'check_2', 'check_3'], axis=1)

        test = frame.iloc[:,-2:]
        test["date"] = test.index
        test["days"] = test["date"].diff()
        test["days"] = test["days"].dt.days
        test["ret"] = test["HGDG_KAPANIS"].pct_change()
        days = test_ = test.loc[test["day"] == "SELL"][["days"]]
        returns = test_ = test.loc[test["day"] == "SELL"][["ret"]]
        returns = returns.join(days)
        returns = returns.dropna()
        returns = return_cum(returns)

    except:
        returns = np.nan
    
    return returns

In [1604]:
MA_days = [7, 14, 20, 50, 100, 200]
start_date = "31-03-2018"
end_date = "31-03-2019"
RSI_day = 14
trajectory_day = 7

In [1636]:
stocks = sm.filter_financials(["turnover", "ebitda", "equity", "gross_profit_margin"], 12, 3, 3)

Reference year to move back: 2016/12
Reference year to move back: 2016/12
Reference year to move back: 2016/12
Reference year to move back: 2016/12


In [1637]:
stocks_vol = get_volatility(stocks, start_date, end_date)
stocks_vol.sort_values(by=["volatility"])

,stock_name,volatility
3,PEKGY,0.004133
10,YGGYO,0.010086
5,TURGG,0.012970
14,SODA,0.013238
16,ARCLK,0.014516
15,SISE,0.014830
0,LOGO,0.015403
9,MAVI,0.015628
1,KORDS,0.016986
4,VESBE,0.018527


In [1624]:
list(sm.wealth_index("SISE", start_date, end_date)["Wealth_Index"])[-1]

1.1672499119955284

In [1625]:
frame = whole_frame("SISE", MA_days, start_date, end_date, RSI_day, trajectory_day)
frame

,MA_7,MA_14,MA_20,MA_50,MA_100,MA_200,MA_7_CH,MA_14_CH,MA_20_CH,MA_50_CH,MA_100_CH,MA_200_CH,RSI,Drawdown,marketV_to_bookV,StockValue
2018-05-25,3.936143,4.024386,4.085220,4.443670,4.516014,4.287185,-0.042913,-0.083713,-0.121110,-0.042664,0.014571,0.014714,38.739172,-0.186758,0.701104,3.9121
2018-05-28,3.949514,4.017700,4.065095,4.430006,4.512083,4.287326,-0.033376,-0.080195,-0.123411,-0.045377,0.012198,0.013890,46.031881,-0.155639,0.72794,4.0618
2018-05-29,3.938814,4.004329,4.050120,4.412784,4.506467,4.286530,-0.032195,-0.078461,-0.123101,-0.048820,0.009539,0.012941,42.647290,-0.184825,0.702781,3.9214
2018-05-30,3.929457,3.983607,4.035615,4.396312,4.499822,4.286062,-0.026822,-0.077840,-0.120449,-0.052179,0.006700,0.011807,38.169807,-0.190645,0.697749,3.8934
2018-05-31,3.904286,3.966343,4.022595,4.377440,4.492632,4.285602,-0.030810,-0.074966,-0.117889,-0.056362,0.003746,0.010677,39.738473,-0.200042,0.667558,3.8482
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-03-25,6.177071,6.235771,6.298345,6.145190,5.588260,5.260533,-0.018651,-0.025052,0.007501,0.117566,0.063834,0.089066,44.739563,-0.092350,0.846797,6.0818
2019-03-26,6.117686,6.190886,6.253385,6.152346,5.597833,5.269381,-0.023371,-0.033635,-0.006530,0.113971,0.064794,0.089878,33.993887,-0.134197,0.807755,5.8014
2019-03-27,6.021000,6.142543,6.204075,6.154474,5.604795,5.276101,-0.038806,-0.038590,-0.019184,0.109113,0.065217,0.090078,33.175897,-0.171716,0.772752,5.5500
2019-03-28,5.896686,6.096271,6.152830,6.156022,5.610017,5.281805,-0.063402,-0.045007,-0.027732,0.105156,0.065446,0.089668,33.661219,-0.180372,0.764675,5.4920


In [1638]:
stock_name = "ARCLK"
MA_days = [7, 14, 20, 50, 100, 200]
start_date = "31-03-2017"
end_date = "31-03-2018"
RSI_day = 14
trajectory_day = 7

In [1649]:
frame = whole_frame(stock_name, MA_days, start_date, end_date, RSI_day, trajectory_day)

In [1650]:
frame['day'] = np.where((frame['MA_7'] > frame['MA_50']) & 
                        (frame['MA_7'] > frame['MA_200']) &
                        (frame['MA_200_CH'] > 0), "BUY", "SELL")

In [1651]:
frame["check"] = frame["day"].shift()

In [1655]:
frame

,MA_7,MA_14,MA_20,MA_50,MA_100,MA_200,MA_7_CH,MA_14_CH,MA_20_CH,MA_50_CH,MA_100_CH,MA_200_CH,RSI,Drawdown,marketV_to_bookV,StockValue,day
2017-08-03,25.037857,25.501157,25.360235,24.9919,23.452712,21.567067,-0.035687,0.021317,0.013853,0.065023,0.073211,0.052903,22.515352,-0.069066,2.75183,24.4587,SELL


In [1653]:
first_oberv = frame["day"][0]
frame["check"][0] = first_oberv

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [1654]:
frame["check"] = frame["day"] == frame["check"] 
frame["check_2"] = frame["check"].shift(-1)
frame["check_3"] = frame["check"] == frame["check_2"] 
frame['check_3'] = np.where(frame["check_3"] == False, "ok", "nope")
frame = frame[frame["check_3"] == "ok"]
frame = frame.loc[frame["day"] == "BUY"]
frame['day'] = np.where(frame['check'] == False, "BUY", "SELL")
frame = frame.drop(['check', 'check_2', 'check_3'], axis=1)

In [1644]:
test = frame.iloc[:,-2:]
test["date"] = test.index
test["days"] = test["date"].diff()
test["days"] = test["days"].dt.days
test["ret"] = test["StockValue"].pct_change()
days = test_ = test.loc[test["day"] == "SELL"][["days"]]
returns = test_ = test.loc[test["day"] == "SELL"][["ret"]]
returns = returns.join(days)
returns = returns.dropna()

In [1645]:
returns

,ret,days


In [1646]:
def return_cum(returns):
    returns["product"] = returns["ret"] + 1
    print(returns["product"].cumprod()[-1])

In [1647]:
return_cum(returns)

IndexError: index out of bounds

In [1498]:
frame

,MA_7,MA_14,MA_20,MA_50,MA_100,MA_200,MA_7_CH,MA_14_CH,MA_20_CH,MA_50_CH,MA_100_CH,MA_200_CH,RSI,Drawdown,marketV_to_bookV,HGDG_KAPANIS,day
2018-06-29,3.740129,3.679586,3.680150,3.588366,3.564181,3.473584,0.021283,-3.773016e-03,-0.003432,0.021247,0.004608,0.000155,75.778331,-0.037606,1.03763,3.7415,BUY
2018-07-12,3.655214,3.699064,3.692800,3.636106,3.565485,3.495563,-0.018685,8.535773e-03,-0.000138,0.027953,-0.002810,0.006971,23.086030,-0.072691,0.993207,3.6051,SELL
2018-07-26,3.705357,3.664979,3.678195,3.697688,3.580024,3.512692,0.022671,-9.214686e-03,0.000530,0.032260,0.004269,0.012543,53.334473,-0.045117,1.02273,3.7123,BUY
2018-08-06,3.692843,3.699100,3.674800,3.690090,3.592453,3.525731,-0.009704,1.200238e-02,-0.003428,0.020479,0.007779,0.016329,50.773359,-0.065154,1.00126,3.6344,SELL
2018-08-14,3.687257,3.695614,3.698190,3.678008,3.600755,3.531209,0.001509,3.022331e-03,0.005304,0.006668,0.009613,0.015447,53.565536,-0.020063,1.04958,3.8097,BUY
2018-08-15,3.691429,3.692136,3.696240,3.676644,3.602509,3.532038,0.009513,2.220446e-16,0.004507,0.003992,0.010366,0.015111,46.153846,-0.057643,1.00931,3.6636,SELL
2019-03-11,3.840371,3.694214,3.579795,3.288460,3.081896,3.282567,0.051449,1.111565e-01,0.104797,0.105907,0.027615,0.000446,79.522691,0.000000,1.00826,3.9072,BUY
2019-08-02,4.628571,4.606429,4.573500,4.618400,4.249944,3.658759,-0.000925,1.160784e-02,-0.028259,0.051117,0.071900,0.063918,58.024691,-0.080321,0.995099,4.5800,SELL
2019-08-23,4.645714,4.595714,4.610000,4.636200,4.340564,3.753516,0.035010,-4.641089e-03,0.008753,0.022180,0.052725,0.062703,59.259259,-0.028112,1.05159,4.8400,BUY
2020-03-03,6.661857,6.881164,7.022320,6.610002,6.120101,5.437350,-0.040442,-5.241803e-02,0.027569,0.075904,0.083385,0.092901,33.490297,-0.135306,1.34819,6.5613,SELL


In [1499]:
test

,HGDG_KAPANIS,day,date,days,ret
2018-06-29,3.7415,BUY,2018-06-29,NaN,NaN
2018-07-12,3.6051,SELL,2018-07-12,13.0,-0.036456
2018-07-26,3.7123,BUY,2018-07-26,14.0,0.029736
2018-08-06,3.6344,SELL,2018-08-06,11.0,-0.020984
2018-08-14,3.8097,BUY,2018-08-14,8.0,0.048234
2018-08-15,3.6636,SELL,2018-08-15,1.0,-0.038349
2019-03-11,3.9072,BUY,2019-03-11,208.0,0.066492
2019-08-02,4.5800,SELL,2019-08-02,144.0,0.172195
2019-08-23,4.8400,BUY,2019-08-23,21.0,0.056769
2020-03-03,6.5613,SELL,2020-03-03,193.0,0.355640
